In [1]:
%run testcong.py
%run XE7

### Read in (or compute) the has table of $a_p\pmod{7}$ for $30$ primes over $400000$.  Computing this takes about 45 minutes

In [2]:
try:
    hashtab7_50 = load('hashtab7_50')
    hashtab7_50_all = load('hashtab7_50_all')
except IOError:
    hashtab7_50_all = make_hash(7,11,500000,50)
    save(hashtab7_50_all, 'hashtab7_50_all')
    hashtab7_50 = dict([(k,v) for k,v in hashtab7_50_all.items() if len(v)>1])
    save(hashtab7_50, 'hashtab7_50')
len(hashtab7_50)    

510g1
862a1
1155i1
1464a1
1755f1
2046c1
2337c1
2597c1
2850w1
3138e1
3392p1
3660b1
3906g1
4167a1
4424a1
4680e1
4928bb1
5184d1
5436a1
5680h1
5910a1
6138g1
6381a1
6594r1
6840b1
7067a1
7325c1
7575h1
7812f1
8064p1
8300a1
8549c1
8789d1
9030bd1
9270v1
9510p1
9768e1
9994a1
10224l1
10465a1
10703d1
10913c1
11152l1
11389a1
11622a1
11856x1
12090bk1
12321e1
12558a1
12810v1
13050f1
13256a1
13485i1
13725a1
13950bt1
14175z1
14425b1
14658i1
14892j1
15120k1
15344b1
15584i1
15808b1
16048bc1
16296c1
16533d1
16731c1
16980c1
17220c1
17442k1
17664b1
17880m1
18135e1
18333h1
18564c1
18800bs1
19032o1
19264i1
19491g1
19698d1
19950c1
20175a1
20400cv1
20650o1
20850e1
21075b1
21294ch1
21525i1
21735e1
21987d1
22218bd1
22464i1
22695i1
22906e1
23136h1
23352h1
23600a1
23816h1
24050q1
24273l1
24490f1
24735b1
24961a1
25200u1
25408c1
25608f1
25861e1
26082q1
26325b1
26550cm1
26802a1
27027r1
27225bw1
27475d1
27720e1
27930t1
28152g1
28336bl1
28566z1
28798a1
29015d1
29238i1
29435c1
29673f1
29901d1
30144y1
30350h1
30576n1
3081

215424i1
215634c1
215880k1
216137a1
216370a1
216549c1
216752o1
216980b1
217200dx1
217413bd1
217674a1
217854co1
218064q1
218320d1
218560a1
218785h1
218988l1
219198l1
219450ea1
219660f1
219921a1
220138a1
220372b1
220608bs1
220825c1
221056c1
221265b1
221502n1
221760ea1
221920b1
222144cb1
222400f1
222630g1
222816b1
223075b1
223278j1
223497f1
223686bp1
223930a1
224181c1
224400id1
224640i1
224880bb1
225144j1
225354r1
225600cb1
225822a1
226080g1
226330e1
226575f1
226800ca1
227010e1
227205d1
227443b1
227690b1
227910c1
228150ch1
228354f1
228600d1
228800dj1
229026bt1
229264f1
229488h1
229746bf1
229950ff1
230202i1
230434n1
230682a1
230913b1
231179c1
231360dt1
231616bq1
231856a1
232112d1
232350e1
232596k1
232830be1
233046cb1
233310bz1
233576g1
233800t1
234048k1
234240bc1
234465bq1
234696u1
234945g1
235200k1
235248dl1
235470bi1
235710j1
235950bv1
236130j1
236398c1
236640by1
236880ej1
237120ck1
237344a1
237600q1
237825l1
238050dc1
238266w1
238459c1
238702d1
238950bc1
239184r1
239424ca1
239725a1
2399

421248f1
421449l1
421694n1
421950q1
422211d1
422478o1
422730ev1
423024f1
423234g1
423381b1
423654bb1
423882a1
424128co1
424350dr1
424554i1
424800dv1
425025r1
425270g1
425518b1
425730n1
425975a1
426271b1
426503b1
426770b1
426944bc1
427200fg1
427442d1
427700bc1
427947b1
428190l1
428400iw1
428632a1
428848o1
429084j1
429284c1
429534r1
429760by1
429990d1
430276a1
430525a1
430771a1
430983b1
431210h1
431433c1
431680w1
431928b1
432150g1
432425m1
432618c1
432848d1
433075b1
433280t1
433530k1
433770a1
434040a1
434283f1
434511y1
434775bs1
434976q1
435231c1
435462b1
435600um1
435897b1
436128bi1
436392m1
436632r1
436800qi1
437038c1
437310bs1
437570h1
437825e1
438075c1
438277d1
438528a1
438758a1
439050p1
439285g1
439530ef1
439758l1
439950ba1
440216a1
440496cv1
440674a1
440949a1
441210t1
441408fk1
441650bn1
441881d1
442134y1
442298g1
442560bd1
442800hd1
443040bc1
443300x1
443520gx1
443760bo1
444031a1
444258g1
444534b1
444738d1
444984f1
445200fe1
445446f1
445680w1
445914h1
446160es1
446400q1
446520bm1


23735

In [11]:
k=list(hashtab7_50_all.keys())[0]
hashtab7_50_all[k]
[next(cc for cc in hashtab7_50_all.values() if '406598{}1'.format(cl) in cc) for cl in ['a','b','c','d']]

[['406598a1'], ['406598b1'], ['406598c1'], ['406598d1']]

### For each set of size ${}>1$, use Kraus's criterion to prove that the curves really are $7$-congruent.  Note that  we only test $p<10^7$ even if Kraus needs more, so the cases where a warning is output should be run again (this was done once outside this notebook).  NB This takes a long time.

In [70]:
bad_pairs = []
for s in hashtab7_50.values():
     if len(s)>1:
         E1 = EllipticCurve(s[0])
         for r in s[1:]:
             E2 = EllipticCurve(r)
             res, info = test_cong(7,E1,E2, mumax=3*10^7)
             if not res:
                 report(res,info,7,s[0],r)
                 bad_pairs.append([s[0],r])


<string>:106: RuntimeWarning: cypari2 leaked 1688 bytes on the PARI stack


KeyboardInterrupt: 

## The result of the previous cell is that there are no bad pairs:

In [3]:
bad_pairs = [] # since the previous cell takes hours
bad_pairs

[]

## We extract all the sets of size greater than 1:

In [4]:
isom_sets = [s for s in hashtab7_50.values() if len(s)>1]
len(isom_sets)

23735

## Separate out the reducible and irreducible sets:

In [5]:
isom_sets_red = []
isom_sets_irred = []
for s in isom_sets:
    isom_sets_irred.append(s) if test_irred(s) else isom_sets_red.append(s)
print("{} irreducible sets, {} reducible sets".format(len(isom_sets_irred),len(isom_sets_red)))

23448 irreducible sets, 287 reducible sets


## Count the numbers of sets of irreducible and reducible congruent classes, and the sizes of these sets:

In [6]:
from collections import Counter

irred_sizes = Counter()

for s in isom_sets_irred:
    irred_sizes[len(s)] += 1
irred_sizes

Counter({2: 21653, 3: 1502, 4: 283, 5: 10})

### So most irreducible sets have size 2 and the maximum size is 5

In [7]:
red_sizes = Counter()
for s in isom_sets_red:
    red_sizes[len(s)] += 1
red_sizes

Counter({2: 96, 3: 79, 4: 32, 5: 24, 7: 12, 10: 7, 6: 6, 11: 6, 8: 4, 9: 4, 19: 4, 12: 2, 15: 2, 21: 2, 33: 1, 80: 1, 17: 1, 50: 1, 22: 1, 36: 1, 38: 1})

In [8]:
max(_)

80

### So reducible sets have size up to 80.

Find all the j-invariants of curves with congruences (irreducibles only):

In [9]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [10]:
irred_js = Set()
JS = irred_js_by_size = {2:Set(), 3:Set(), 4:Set(),5:Set()}
for s in isom_sets_irred:
    js = set_js(s)
    irred_js = irred_js + Set(js)
    irred_js_by_size[len(s)] += Set(js)

In [11]:
len(irred_js), [(n,len(irred_js_by_size[n])) for n in irred_js_by_size.keys()]

(11761, [(2, 11135), (3, 1455), (4, 340), (5, 36)])

In [15]:
len([j for j in irred_js_by_size[2] if not j in sum([irred_js_by_size[n] for n in [3,4,5]],Set())])

10180

In [16]:
[len([j for j in irred_js_by_size[3] if j in irred_js_by_size[n]]) for n in [4,5]]

[226, 17]

In [17]:
[len([j for j in irred_js_by_size[4] if j in irred_js_by_size[n]]) for n in [5]]

[21]

In [18]:
[n for n in [2,3,4] if 1728 in irred_js_by_size[n]]

[3]

In [19]:
sets3 = [s for s in isom_sets_irred if len(s)==3]
len(sets3)

1502

In [20]:
[s for s in sets3 if 1728 in set_js(s)]

[['576f1', '65088cm1', '65088cn1'],
 ['576g1', '65088cu1', '65088cv1'],
 ['288a1', '32544b1', '32544c1'],
 ['288e1', '32544bd1', '32544be1']]

In [34]:
test7iso(['288a1', '32544b1', '32544c1'])

[['288a1', '288a2', '32544b1', '32544c1'], []]

In [35]:
test7iso(['288e1', '32544bd1', '32544be1'])

[['288e1', '288e2', '32544bd1', '32544be1'], []]

In [36]:
test7iso(['576f1', '65088cm1', '65088cn1'])

[['576f1', '576f2', '65088cm1', '65088cn1'], []]

In [37]:
test7iso(['576g1', '65088cu1', '65088cv1'])

[['576g2', '576g1', '65088cu1', '65088cv1'], []]

In [12]:
jdict = {}
for s in isom_sets_irred:
    js = dict([(lab,class_js(lab)) for lab in s])
    for lab1 in s:
        E0 = EllipticCurve(lab1)
        other_j = sorted(list(sum([js[lab2] for lab2 in s if lab2!=lab1], Set())))
        for E in E0.isogeny_class():
            j = E.j_invariant()
            jdict[j] = other_j
        

In [13]:
len(jdict)

11761

In [72]:
len(isom_sets_irred), len(isom_sets_red)

(23448, 287)

## Count sizes of reducible sets:

In [71]:
for s in isom_sets_red:
    red_sizes[len(s)] += 1
red_sizes  

Counter({2: 288, 3: 237, 4: 96, 5: 72, 7: 36, 10: 21, 6: 18, 11: 18, 8: 12, 9: 12, 19: 12, 12: 6, 15: 6, 21: 6, 33: 3, 80: 3, 17: 3, 50: 3, 22: 3, 36: 3, 38: 3})

### Reducible sets have various sizes up to 80

## Next we go through all the irreducible sets, taking all the curves in each isogeny class and partitioning them into 1 or 2 subsets such that curves in the same subset are symplectically isomorphic, while curves in different subsets are antisymplectically isomorphic.

### The code which does the work is in testcong.py

 all_c is a function defined in testcong.py which, given the label of one curve E in a 7-irreducible isogeny class,  return the list of all labels of curves in that class, partitioned    into 2 parts where those in the first part are m-isogenous to E           with legendre(m,7)=+1 and those in the second part (which may be   empty) are m-isogenous to E with legendre(m,7)=-1.
 
 The items in the lists output are pairs (curve label, isogeny degree)

In [16]:
[all_c(c) for c in isom_sets_red[0]]

[[('149058fn1', 1), ('149058fn2', 7)],
 [('149058ft1', 1), ('149058ft2', 7)],
 [('149058fw1', 1), ('149058fw2', 7)]]

In [17]:
[[all_c(c) for c in s] for s in isom_sets_irred[:10]]

[[[('78400lc1', 1)], [('235200e1', 1)]],
 [[('9408co1', 1), ('9408co2', 3)], [('9408de1', 1), ('9408de2', 2)]],
 [[('4012a1', 1)], [('132396a1', 1)]],
 [[('35322bb1', 1)], [('35322bc1', 1), ('35322bc2', 2)]],
 [[('462400r1', 1)], [('462400cd1', 1)]],
 [[('121104br1', 1)], [('121104bs1', 1)]],
 [[('704f1', 1), ('704f2', 3)], [('16192s1', 1)], [('164032s1', 1)]],
 [[('158760cp1', 1)], [('158760cz1', 1)]],
 [[('3610c1', 1)], [('10830d1', 1), ('10830d2', 2)]],
 [[('90944ct1', 1)], [('90944du1', 1)]]]

### Test one set, containing 2 isogeny classes:

In [18]:
isom_sets_irred[3]

['35322bb1', '35322bc1']

### The function test7iso() takes such a set of isogeny classes (of any size) and returns a list of length 2 lists of curve labels such that the curves in each sublist are symplectically 7-isogenous.  The second list may be empty if all the isomorphisms are symplectic, as in this example:

In [19]:
test7iso(isom_sets_irred[3])

[['35322bb1', '35322bc2', '35322bc1'], []]

### In this second example we see both symplectic and antisymplectic isomorphisms:

In [20]:
test7iso(isom_sets_irred[980])

[['6336cl1', '6336cn1', '6336cn2'], []]

### The preceding output means that '100800hc1' and '100800hc2' are symplectically isomorphic (in fact they are $2$-isogenous and $2$ is a quadratic non-residue) while '100800gw1' is anti-symplectically isomorphic to each.

In [21]:
all_c('100800hc1')

[('100800hc1', 1), ('100800hc2', 2)]

In [22]:
(test_isom_labels('100800hc1','100800hc2'),
test_isom_labels('100800gw1','100800hc1'),
test_isom_labels('100800gw1','100800hc2'))

(True, False, False)

In [23]:
(test_isom_labels('100800hc1','100800hc2', symplectic=False),
test_isom_labels('100800gw1','100800hc1', symplectic=False),
test_isom_labels('100800gw1','100800hc2', symplectic=False))

(False, True, True)

## Here follows the long run which takes several hours

In [28]:
def test7(lab1, lab2):
    E1 = EllipticCurve(lab1)
    E2 = EllipticCurve(lab2)
    res1 = test_isom(E1,E2,True)
    res2 = test_isom(E1,E2,False)
    if res1 and res2:
        print("Problem: curves {} and {} are both symplectically and anti-sympectically isomorphic!".format(lab1,lab2))
        return 0
    elif not (res1 or res2):
        print("Problem: curves {} and {} are neither symplectically nor anti-sympectically isomorphic!".format(lab1,lab2))
        return 0
    else:
        return +1 if res1 else -1

In [ ]:
nsets = len(isom_sets_irred)
irred_pairs_symp = []
irred_pairs_anti = []
irred_pairs_none = []
irred_pairs_bad = []
for i,s in enumerate(isom_sets_irred):
    print("Set #{}/{} with {} classes ({:.3f}%)".format(i+1,nsets,len(s),float(100.0*(i+1)/nsets)))
    #sys.stdout.write('\r')
    sys.stdout.flush()
    for i1,lab1 in enumerate(s):
        for i2, lab2 in enumerate(s):
            if i1<i2:
                try:
                    res = test7(lab1,lab2)
                    if res==+1:
                        sres = "symplectic"
                        irred_pairs_symp.append([lab1,lab2])
                    elif res==-1:
                        sres = "anti-symplectic"
                        irred_pairs_anti.append([lab1,lab2])
                    elif res==0:
                        sres = "neither"
                        irred_pairs_none.append([lab1,lab2])
                    else:
                        sres = "**********?????????*************"
                        irred_pairs_bad.append([lab1,lab2])
                    print("{} and {}: {}".format(lab1,lab2,sres))
                except:
                    print("error processing set number {}: {} and {}".format(i,lab1,lab2))

Set #1/23448 with 2 classes (0.004%)
78400lc1 and 235200e1: symplectic
Set #2/23448 with 2 classes (0.009%)
9408co1 and 9408de1: anti-symplectic
Set #3/23448 with 2 classes (0.013%)
4012a1 and 132396a1: anti-symplectic
Set #4/23448 with 2 classes (0.017%)
35322bb1 and 35322bc1: symplectic
Set #5/23448 with 2 classes (0.021%)
462400r1 and 462400cd1: symplectic
Set #6/23448 with 2 classes (0.026%)
121104br1 and 121104bs1: anti-symplectic
Set #7/23448 with 3 classes (0.030%)
704f1 and 16192s1: symplectic
704f1 and 164032s1: symplectic
16192s1 and 164032s1: symplectic
Set #8/23448 with 2 classes (0.034%)
158760cp1 and 158760cz1: symplectic
Set #9/23448 with 2 classes (0.038%)
3610c1 and 10830d1: symplectic
Set #10/23448 with 2 classes (0.043%)
90944ct1 and 90944du1: symplectic
Set #11/23448 with 2 classes (0.047%)
9042g1 and 40278q1: symplectic
Set #12/23448 with 2 classes (0.051%)
738e1 and 247230cf1: symplectic
Set #13/23448 with 2 classes (0.055%)
318864g1 and 416976f1: symplectic
Set #

397488w1 and 397488ff1: symplectic
Set #105/23448 with 3 classes (0.448%)
7050j1 and 7050l1: symplectic
7050j1 and 345450du1: anti-symplectic
7050l1 and 345450du1: anti-symplectic
Set #106/23448 with 2 classes (0.452%)
145530bh1 and 145530bo1: anti-symplectic
Set #107/23448 with 2 classes (0.456%)
382200gp1 and 382200jm1: symplectic
Set #108/23448 with 2 classes (0.461%)
87150r1 and 87150s1: symplectic
Set #109/23448 with 3 classes (0.465%)
648c1 and 12312i1: symplectic
648c1 and 74520h1: symplectic
12312i1 and 74520h1: symplectic
Set #110/23448 with 2 classes (0.469%)
451350v1 and 451350cm1: symplectic
Set #111/23448 with 2 classes (0.473%)
97650dm1 and 265050di1: anti-symplectic
Set #112/23448 with 2 classes (0.478%)
16606s1 and 249090cz1: anti-symplectic
Set #113/23448 with 2 classes (0.482%)
74008i1 and 222024j1: symplectic
Set #114/23448 with 2 classes (0.486%)
7938ba1 and 230202bc1: anti-symplectic
Set #115/23448 with 2 classes (0.490%)
435120fg1 and 435120gq1: symplectic
Set #11

13590l1 and 231030y1: symplectic
Set #203/23448 with 2 classes (0.866%)
23925ba1 and 311025br1: symplectic
Set #204/23448 with 2 classes (0.870%)
30400bx1 and 91200fy1: symplectic
Set #205/23448 with 2 classes (0.874%)
9072b1 and 45360m1: symplectic
Set #206/23448 with 2 classes (0.879%)
19110s1 and 19110y1: symplectic
Set #207/23448 with 2 classes (0.883%)
379050k1 and 379050z1: anti-symplectic
Set #208/23448 with 3 classes (0.887%)
1485b1 and 72765u1: symplectic
1485b1 and 381645i1: symplectic
72765u1 and 381645i1: symplectic
Set #209/23448 with 2 classes (0.891%)
27200cf1 and 81600fk1: anti-symplectic
Set #210/23448 with 3 classes (0.896%)
2040e1 and 14280q1: symplectic
2040e1 and 108120e1: symplectic
14280q1 and 108120e1: symplectic
Set #211/23448 with 2 classes (0.900%)
60720cx1 and 376464br1: symplectic
Set #212/23448 with 2 classes (0.904%)
86394bt1 and 86394bu1: anti-symplectic
Set #213/23448 with 2 classes (0.908%)
90629i1 and 220099a1: symplectic
Set #214/23448 with 2 classes

7920bj1 and 229680k1: symplectic
Set #298/23448 with 2 classes (1.271%)
10800cd1 and 183600eg1: symplectic
Set #299/23448 with 2 classes (1.275%)
5530b1 and 171430x1: symplectic
Set #300/23448 with 2 classes (1.279%)
64288a1 and 321440h1: symplectic
Set #301/23448 with 2 classes (1.284%)
1856k1 and 102080bj1: symplectic
Set #302/23448 with 2 classes (1.288%)
84966b1 and 84966bf1: anti-symplectic
Set #303/23448 with 2 classes (1.292%)
2832g1 and 36816t1: symplectic
Set #304/23448 with 2 classes (1.296%)
29512c1 and 29512d1: symplectic
Set #305/23448 with 2 classes (1.301%)
160083ci1 and 160083cp1: symplectic
Set #306/23448 with 2 classes (1.305%)
210450o1 and 210450p1: symplectic
Set #307/23448 with 2 classes (1.309%)
23856y1 and 262416bf1: symplectic
Set #308/23448 with 2 classes (1.314%)
2204a1 and 271092i1: symplectic
Set #309/23448 with 2 classes (1.318%)
8820v1 and 8820w1: anti-symplectic
Set #310/23448 with 2 classes (1.322%)
14094e1 and 183222m1: symplectic
Set #311/23448 with 2 

5550e1 and 75850f1: symplectic
Set #402/23448 with 3 classes (1.714%)
3312r1 and 36432bw1: symplectic
3312r1 and 202032q1: anti-symplectic
36432bw1 and 202032q1: anti-symplectic
Set #403/23448 with 2 classes (1.719%)
14124e1 and 268356c1: symplectic
Set #404/23448 with 2 classes (1.723%)
26160i1 and 99408a1: symplectic
Set #405/23448 with 2 classes (1.727%)
3024j1 and 15120l1: symplectic
Set #406/23448 with 2 classes (1.731%)
29670bc1 and 290766cn1: symplectic
Set #407/23448 with 4 classes (1.736%)
74800bz1 and 278800k1: anti-symplectic
74800bz1 and 278800v1: symplectic
74800bz1 and 360400e1: symplectic
278800k1 and 278800v1: anti-symplectic
278800k1 and 360400e1: anti-symplectic
278800v1 and 360400e1: symplectic
Set #408/23448 with 2 classes (1.740%)
227430br1 and 227430cc1: anti-symplectic
Set #409/23448 with 2 classes (1.744%)
2562l1 and 371490ce1: symplectic
Set #410/23448 with 2 classes (1.749%)
2808n1 and 14040bi1: symplectic
Set #411/23448 with 2 classes (1.753%)
363258e1 and 36

...

14175i1 and 411075bw1: symplectic
Set #1331/23448 with 2 classes (5.676%)
84672ga1 and 84672gb1: symplectic
Set #1332/23448 with 2 classes (5.681%)
345450h1 and 345450ca1: symplectic
Set #1333/23448 with 2 classes (5.685%)
354a1 and 63366b1: symplectic
Set #1334/23448 with 2 classes (5.689%)
11600k1 and 266800j1: symplectic
Set #1335/23448 with 2 classes (5.693%)
82786j1 and 145266b1: symplectic
Set #1336/23448 with 2 classes (5.698%)
10432f1 and 365120h1: symplectic
Set #1337/23448 with 2 classes (5.702%)
2548c1 and 43316n1: symplectic
Set #1338/23448 with 2 classes (5.706%)
95550fs1 and 95550ft1: symplectic
Set #1339/23448 with 2 classes (5.711%)
92169g1 and 460845cj1: symplectic
Set #1340/23448 with 2 classes (5.715%)
44400bz1 and 162800bv1: symplectic
Set #1341/23448 with 2 classes (5.719%)
176400oj1 and 176400qs1: symplectic
Set #1342/23448 with 2 classes (5.723%)
7938e1 and 87318s1: anti-symplectic
Set #1343/23448 with 2 classes (5.728%)
239148s1 and 312732h1: symplectic
Set #134

17010i1 and 391230z1: symplectic
Set #1435/23448 with 2 classes (6.120%)
2928i1 and 67344p1: symplectic
Set #1436/23448 with 2 classes (6.124%)
34320bf1 and 34320bg1: symplectic
Set #1437/23448 with 2 classes (6.128%)
23534w1 and 117670q1: anti-symplectic
Set #1438/23448 with 2 classes (6.133%)
13386h1 and 66930p1: symplectic
Set #1439/23448 with 2 classes (6.137%)
56448e1 and 282240r1: symplectic
Set #1440/23448 with 2 classes (6.141%)
24321f1 and 413457g1: symplectic
Set #1441/23448 with 2 classes (6.146%)
612b1 and 29988r1: anti-symplectic
Set #1442/23448 with 2 classes (6.150%)
13248d1 and 13248e1: symplectic
Set #1443/23448 with 2 classes (6.154%)
43200jr1 and 475200f1: anti-symplectic
Set #1444/23448 with 2 classes (6.158%)
4575d1 and 224175bp1: symplectic
Set #1445/23448 with 2 classes (6.163%)
7704t1 and 84744k1: symplectic
Set #1446/23448 with 2 classes (6.167%)
18928d1 and 321776bw1: symplectic
Set #1447/23448 with 2 classes (6.171%)
22050fn1 and 374850pd1: symplectic
Set #14

720i1 and 33840cg1: symplectic
Set #1541/23448 with 2 classes (6.572%)
44650d1 and 312550l1: symplectic
Set #1542/23448 with 2 classes (6.576%)
25992u1 and 285912m1: symplectic
Set #1543/23448 with 2 classes (6.581%)
379456hw1 and 379456jh1: anti-symplectic
Set #1544/23448 with 2 classes (6.585%)
30996k1 and 30996l1: anti-symplectic
Set #1545/23448 with 2 classes (6.589%)
202800bp1 and 202800cv1: symplectic
Set #1546/23448 with 2 classes (6.593%)
1968b1 and 482160fz1: symplectic
Set #1547/23448 with 2 classes (6.598%)
419628bd1 and 419628bj1: anti-symplectic
Set #1548/23448 with 2 classes (6.602%)
146224u1 and 238576q1: symplectic
Set #1549/23448 with 2 classes (6.606%)
223587bw1 and 223587cb1: symplectic
Set #1550/23448 with 3 classes (6.610%)
1890i1 and 35910w1: anti-symplectic
1890i1 and 126630w1: symplectic
35910w1 and 126630w1: anti-symplectic
Set #1551/23448 with 3 classes (6.615%)
47328u1 and 64032bd1: symplectic
47328u1 and 136416g1: symplectic
64032bd1 and 136416g1: symplectic

11466k1 and 424242bt1: anti-symplectic
Set #1640/23448 with 2 classes (6.994%)
9408db1 and 272832ey1: anti-symplectic
Set #1641/23448 with 2 classes (6.998%)
5957a1 and 77441c1: symplectic
Set #1642/23448 with 2 classes (7.003%)
247350y1 and 247350z1: symplectic
Set #1643/23448 with 2 classes (7.007%)
33462cn1 and 234234fr1: anti-symplectic
Set #1644/23448 with 2 classes (7.011%)
83655bi1 and 485199j1: anti-symplectic
Set #1645/23448 with 2 classes (7.016%)
16320cu1 and 114240jr1: symplectic
Set #1646/23448 with 2 classes (7.020%)
310050cc1 and 310050ce1: symplectic
Set #1647/23448 with 2 classes (7.024%)
65533a1 and 65533b1: symplectic
Set #1648/23448 with 3 classes (7.028%)
567a1 and 82215g1: symplectic
567a1 and 459837h1: symplectic
82215g1 and 459837h1: symplectic
Set #1649/23448 with 2 classes (7.033%)
40898i1 and 286286z1: symplectic
Set #1650/23448 with 2 classes (7.037%)
84560t1 and 157040f1: symplectic
Set #1651/23448 with 2 classes (7.041%)
16048be1 and 337008c1: symplectic
S

42560f1 and 79040n1: symplectic
Set #1741/23448 with 2 classes (7.425%)
10582e1 and 158730h1: anti-symplectic
Set #1742/23448 with 2 classes (7.429%)
4425f1 and 30975n1: symplectic
Set #1743/23448 with 2 classes (7.433%)
48720cp1 and 132240f1: anti-symplectic
Set #1744/23448 with 2 classes (7.438%)
15162i1 and 75810bg1: anti-symplectic
Set #1745/23448 with 4 classes (7.442%)
184a1 and 3496h1: symplectic
184a1 and 6808f1: symplectic
184a1 and 266984f1: symplectic
3496h1 and 6808f1: symplectic
3496h1 and 266984f1: symplectic
6808f1 and 266984f1: symplectic
Set #1746/23448 with 2 classes (7.446%)
243334r1 and 243334s1: anti-symplectic
Set #1747/23448 with 2 classes (7.451%)
2835e1 and 252315n1: symplectic
Set #1748/23448 with 2 classes (7.455%)
110160cp1 and 110160cq1: anti-symplectic
Set #1749/23448 with 2 classes (7.459%)
4752b1 and 337392e1: symplectic
Set #1750/23448 with 2 classes (7.463%)
141120cx1 and 141120ej1: symplectic
Set #1751/23448 with 2 classes (7.468%)
57800q1 and 173400e

In [ ]:
[len(t) for t in [irred_pairs_symp,irred_pairs_anti,irred_pairs_none,irred_pairs_bad]]

In [33]:
isom_sets_irred_symplectic2 = []
nsets = len(isom_sets_irred[:20])
for i,s in enumerate(isom_sets_irred[:20]):
    if i%20==19:
        print("Set #{}/{} ({:.3f}%)".format(i+1,nsets,float(100.0*(i+1)/nsets)))
    sys.stdout.write('\r')
    sys.stdout.flush()
    try:
        res = test7iso(s)
        isom_sets_irred_symplectic2.append(res)
        #print(res)
    except:
        print("error processing set number {}: {}".format(i,s))

Set #20/20 (100.000%)


In [22]:
[(i,s1,s2) for i,s1,s2 in zip(range(len(isom_sets_irred_symplectic2)), isom_sets_irred_symplectic, isom_sets_irred_symplectic2) if s1!=s2]

NameError: name 'isom_sets_irred_symplectic2' is not defined

In [25]:
len(isom_sets_irred_symplectic2)

19883

In [26]:
out = open("mod7irreducible_congruences.txt",'w')
for ss in isom_sets_irred_symplectic2:
    out.write(str(ss))
    out.write("\n")
out.close()    

In [27]:
isom_sets_irred_anti = [ss for ss in isom_sets_irred_symplectic2 if ss[1]]
len(isom_sets_irred_anti), len(isom_sets_irred_symplectic2)-len(isom_sets_irred_anti)

(7489, 12394)

In [25]:
from sage.databases.cremona import parse_cremona_label
parse_cremona_label("11a1")

(11, 'a', 1)

In [26]:
def class_label(label):
    return "".join([str(s) for s in parse_cremona_label(label)[:2]])

In [27]:
class_label("11a1")

'11a'

In [28]:
def non_triv(ss):
    classes = [[class_label(s) for s in si] for si in ss]
    return not any(c in classes[1] for c in classes[0])

In [32]:
isom_sets_irred_nontriv = [ss for ss in isom_sets_irred_anti if non_triv(ss)]
len(isom_sets_irred_nontriv)

5092

In [33]:
out = open("mod7_antipairs.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_nontriv[:-1]:
    print(s[0],s[1])
    out.write('["{}", "{}"],\\\n'.format(s[0][0],s[1][0]))
s = isom_sets_irred_nontriv[-1]
out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

(['18326z1'], ['238238ba1'])
(['100800gw1'], ['100800hc1', '100800hc2'])
(['1178b1'], ['133114h1'])
(['1872r1', '1872r2', '1872r3', '1872r4'], ['87984bs1', '91728ft1'])
(['159936jh1'], ['159936jq1'])
(['204309p1'], ['364203v1'])
(['66240en1'], ['66240es1'])
(['9360bk1'], ['271440ch1'])
(['6534f1'], ['45738v1', '280962z1'])
(['17974i1'], ['17974j1'])
(['24780a1'], ['338660f1'])
(['28224et1'], ['28224gc1', '28224gc2'])
(['105105bb1'], ['105105bc1'])
(['87210w1'], ['87210x1'])
(['39984ct1'], ['39984do1'])
(['94640df1'], ['283920di1'])
(['3600br2', '3600br1'], ['46800ev1', '61200gs1'])
(['176400dm1'], ['176400fq1'])
(['95370dr1'], ['95370ds1'])
(['158025bh1'], ['158025bi1'])
(['144150bw1'], ['144150cn1'])
(['41292g1'], ['41292h1'])
(['19881g1'], ['139167i1'])
(['161994u1'], ['161994bg1'])
(['18928be1'], ['56784dc1'])
(['2704l1'], ['8112bf1'])
(['9675j1'], ['396675q1'])
(['3042a2', '3042a1'], ['57798d1'])
(['280476bd1'], ['280476be1'])
(['9702o1', '9702o2'], ['300762cn1'])
(['22050g1'], ['2

(['1950h1'], ['142350bp1'])
(['322896h1', '322896h2'], ['322896i2', '322896i1'])
(['122304gr1'], ['122304ij4', '122304ij2', '122304ij3', '122304ij1'])
(['10150d1'], ['131950ce1'])
(['198a4', '198a1', '198a2', '198a3'], ['4554o1', '110286j1'])
(['34496cd1'], ['103488gz1'])
(['3780c1'], ['223020bk1'])
(['275373l1'], ['275373m1'])
(['163350cx1'], ['163350ee1'])
(['310464ew1'], ['310464fb2', '310464fb1'])
(['16560bl1'], ['182160cr1'])
(['204490ch1'], ['204490cm1'])
(['35772a1'], ['276420f1'])
(['187425fk1'], ['253575fk1'])
(['17550cb1'], ['122850cr1'])
(['286650v1'], ['286650gk1'])
(['61446cc1'], ['102410bs1'])
(['318270p1'], ['318270q1'])
(['127050br1'], ['127050bz1'])
(['8820n2', '8820n1'], ['8820p1'])
(['101454c1'], ['150810l1'])
(['353934bg1', '353934bg2'], ['353934bh1'])
(['14514n1'], ['237062u1'])
(['78400a1'], ['78400s1'])
(['136710gm1'], ['136710gr1'])
(['302400lt1'], ['302400sa1'])
(['17424br1'], ['87120el1'])
(['132940b1'], ['398820bn1'])
(['168432i1'], ['168432j1'])
(['141120iz1

(['91728fs1'], ['91728fw2', '91728fw1'])
(['2800bf1'], ['8400bw1', '8400bw2'])
(['152460x1'], ['152460z1', '152460z2'])
(['344988f1'], ['344988h1'])
(['134946cu1'], ['134946cx1'])
(['1360e2', '1360e1'], ['131920l1'])
(['11466g1', '11466g2'], ['217854en1'])
(['368f1', '54096bn1'], ['18768o1'])
(['2160u1', '153360p1'], ['41040br1'])
(['34914a1'], ['174570da1'])
(['19008br1'], ['95040gw1'])
(['3612g1'], ['299796i1'])
(['6084p1', '6084p2'], ['298116s1'])
(['20286c1'], ['385434ck1'])
(['225302cx1'], ['225302cy1'])
(['154154m1'], ['154154ba1'])
(['4554bh1'], ['159390e2', '159390e1'])
(['282150cx1'], ['282150fe1'])
(['52800fy1'], ['123200ga1'])
(['35490cf1', '35490cf2'], ['291018ch1'])
(['31050bi1'], ['341550eb1'])
(['37926g1', '37926p1'], ['37926r1'])
(['63296o1', '217408e1'], ['244928d1'])
(['2975a1'], ['235025a1'])
(['36300bl1'], ['36300bm1'])
(['170352c1'], ['170352cu1'])
(['282576k3', '282576k2', '282576k1', '282576k6', '282576k5', '282576k4'], ['282576o1'])
(['1650e1', '51150t1'], ['169

In [34]:
out = open("mod7_antipairs_all.m", "w")
out.write('pairs := [\\\n')
for s in isom_sets_irred_symplectic2:
    for s0 in s[0]:
        for s1 in s[1]:
            out.write('["{}", "{}"],\\\n'.format(s0,s1))
#s = isom_sets_irred_symplectic2[-1]
#out.write('["{}", "{}"]\\\n'.format(s[0][0],s[1][0]))
out.write('];\n')
out.close()

In [15]:
%run XE7

In [16]:
R2.<t> = QQ[]

In [104]:
j0sets = [s for s in isom_sets_irred if any([EllipticCurve(lab).j_invariant()==0 for lab in s])]

In [248]:
asq_sets = [isom_sets_irred[i] for i in [22,27,43,63,70,71,76,84]]
for s in asq_sets:
    print(test7iso(s))

[['9016m1', '171304h1', '333592j1'], []]
[['5550bk1', '5550bm4', '5550bm1', '5550bm3', '5550bm2'], []]
[['33856q1', '169280bl1'], []]
[['9800f1', '9800g1'], []]
[['82810b1', '82810g1'], []]
[['40768a1', '40768p1'], []]
[['174048a1', '174048p1'], []]
[['290325bt1', '290325bu1'], []]


## Testing isomorphisms among sets isogeny classes which are reducible and mutually isomorphic up to semisimplification

In [73]:
len(isom_sets_red)

287

In [75]:
isom_sets_red[1]

['507b1',
 '10309d1',
 '16055c1',
 '27885d1',
 '27885f1',
 '78585b1',
 '124215j1',
 '169845m1',
 '262119f1',
 '452751d1']

In [76]:
mod_p_iso_red_set(isom_sets_red[1],7,1)

star field subsets: [['452751d1'], ['124215j2'], ['16055c1'], ['169845m1'], ['10309d1'], ['27885d1'], ['262119f1'], ['27885f1'], ['507b1', '78585b1']]
star-star field subsets: [['27885f2', '169845m2'], ['262119f2'], ['78585b2'], ['507b2', '10309d2'], ['16055c2', '124215j1'], ['27885d2'], ['452751d2']]


([['452751d1'],
  ['124215j2'],
  ['16055c1'],
  ['169845m1'],
  ['10309d1'],
  ['27885d1'],
  ['262119f1'],
  ['27885f1'],
  ['507b1', '78585b1']],
 [['27885f2', '169845m2'],
  ['262119f2'],
  ['78585b2'],
  ['507b2', '10309d2'],
  ['16055c2', '124215j1'],
  ['27885d2'],
  ['452751d2']])

In [27]:
%run testcong.py

In [28]:
isom_sets_red_complete = []
for i, ss in enumerate(isom_sets_red):
    S1, S2 = mod_p_iso_red_set(ss,7,0)
    #print(i, S1, S2)
    nontriv = [s for s in S1+S2 if len(s)>1]
    if nontriv:
        print(nontriv)
        isom_sets_red_complete += nontriv
        

[['507b1', '78585b1'], ['507b2', '10309d2'], ['27885f2', '169845m2'], ['16055c2', '124215j1']]
[['49200cs1', '114800by1'], ['51600de1', '189200c1'], ['19600db2', '58800jb2'], ['5200bc1', '109200gm1', '171600bz1'], ['58800jb1', '135600t2']]
[['32400db2', '356400gn2']]
[['67600bi1', '67600bj1']]
[['99225a1', '99225j2']]
[['24336bk1', '121680dv1']]
[['35574d2', '59290f2'], ['154154cu1', '462462bi1', '462462bl1']]
[['117045w1', '117045x1'], ['117045w2', '257499g2']]
[['10816bk1', '227136t1', '356928fw1']]
[['5184p3', '160704u2']]
[['76050br1', '76050bs1']]
[['161728bk1', '254144ce1']]
[['226512ce1', '226512ch1']]
[['126350c1', '198550ck1']]
[['100672di1', '302016gq1']]
[['84966dm2', '141610c2']]
[['4050bh1', '433350bd1'], ['4050bh3', '125550dh2']]
[['162b1', '17334h1', '293706ci2'], ['162b3', '5022g2'], ['167994c2', '184518g2']]
[['78400go2', '235200bby2']]
[['11025y1', '143325ev2'], ['11025y3', '143325eq2']]
[['87318bg1', '182574bl2']]
[['24336bl1', '170352z1', '267696cl1']]
[['40432o1', 

[['29520bn1', '41328bi1'], ['1872n1', '13104bq1', '20592bl1', '458640by2'], ['7056bs2', '35280eg2'], ['6192o1', '340560s1'], ['7056bs1', '16272y2']]
[['144354g2', '156702bg2', '370146z2'], ['294a1', '33222p2'], ['294a2', '490f2', '61446bl2'], ['1274n1', '3822v1', '19110bm2', '42042cn1'], ['8526o1', '196098bc2'], ['12642z1', '231770co1'], ['4018s1', '60270t1']]
[['7938u1', '293706cg2'], ['7938u3', '246078by2']]
[['9408v2', '15680bl2'], ['40768bc1', '122304dd1']]
[['146205e2', '321651f2']]
[['351310k2', '373498j2']]
[['25920cw1', '440640fd1'], ['25920cw2', '57024cx2', '254016do1']]
[['63504ck1', '317520bm2']]
[['8112bd2', '164944b2']]
[['2064e1', '37840w1'], ['2352l2', '3920bj2', '491568bh2'], ['208d1', '4368n1', '6864j1', '152880dn2'], ['4592f1', '9840r1'], ['164976n2', '179088bh2', '423024l2'], ['2352l1', '5424l2'], ['60240o2', '302640y2']]
[['9386f1', '197106ce1', '309738g1'], ['106134m2', '176890eo2'], ['207214bb1', '444030r1'], ['106134m1', '244758e2']]
[['10816y1', '54080by1'], ['1

In [77]:
len(isom_sets_red_complete)

384

In [78]:
[(n, len([s for s in isom_sets_red_complete if len(s)==n])) for n in [2,3,4]]

[(2, 313), (3, 61), (4, 10)]

In [32]:
Set(len(s) for s in isom_sets_red_complete)

{2, 3, 4}

In [85]:
def iso23(lab):
    # Return list of labels of curves 2- or 3-isogenous to given one.
    # We rely on the reducible isogeny classes having only one 2- or 3-isogeny
    # in addition to the 7-isogeny, so the returned list will contain 1 or 2 curves
    E=EllipticCurve(lab)
    ans=[lab]
    lab2 = [Elabel(phi.codomain()) for phi in E.isogenies_prime_degree(2)]
    if lab2:
        ans += lab2
    else:
        lab3 = [Elabel(phi.codomain()) for phi in E.isogenies_prime_degree(3)]
        ans += lab3
    return ans        

In [93]:
ns=0
n=0
m=0
for ss in isom_sets_red_complete:
    ns+=1
    for s in ss:
        n+=1
        s23 = iso23(s)
        if len(s23)>1:
            m+=1
        #print(s23)
print("processed {} curves in {} sets, of which {} were expanded".format(n,ns,m))            

processed 849 curves in 384 sets, of which 60 were expanded


In [94]:
isom_sets_red_iso23 = [sum([iso23(s) for s in ss],[]) for ss in isom_sets_red_complete]

In [96]:
len(isom_sets_red_iso23)

384

In [97]:
Set(len(s) for s in isom_sets_red_iso23)

{2, 3, 4}

In [99]:
[(n, len([s for s in isom_sets_red_iso23 if len(s)==n])) for n in [2,3,4]]

[(2, 263), (3, 101), (4, 20)]

In [100]:
isom_sets_red_split = []
for i,ss in enumerate(isom_sets_red_iso23):
    E1 = ss[0]
    symps = [E1]
    antis = []
    for E2 in ss[1:]:
        res = test_isom_labels(E1,E2, symplectic=True, verbose=False)
        print("The isomorphism between {} and {} is {}".format(E1,E2,"symplectic" if res else "antisyplectic"))
        (symps if res else antis).append(E2)
    isom_sets_red_split.append([symps,antis])    
    print("{}: {}={}+{}".format(i,len(ss),len(symps),len(antis)))

The isomorphism between 507b1 and 78585b1 is symplectic
0: 2=2+0
The isomorphism between 507b2 and 10309d2 is symplectic
1: 2=2+0
The isomorphism between 27885f2 and 169845m2 is symplectic
2: 2=2+0
The isomorphism between 16055c2 and 124215j1 is symplectic
3: 2=2+0
The isomorphism between 49200cs1 and 114800by1 is symplectic
4: 2=2+0
The isomorphism between 51600de1 and 189200c1 is symplectic
5: 2=2+0
The isomorphism between 19600db2 and 58800jb2 is antisyplectic
6: 2=1+1
The isomorphism between 5200bc1 and 109200gm1 is symplectic
The isomorphism between 5200bc1 and 171600bz1 is antisyplectic
7: 3=2+1
The isomorphism between 58800jb1 and 135600t2 is symplectic
8: 2=2+0
The isomorphism between 32400db2 and 356400gn2 is symplectic
9: 2=2+0
The isomorphism between 67600bi1 and 67600bj1 is antisyplectic
10: 2=1+1
The isomorphism between 99225a1 and 99225j2 is symplectic
11: 2=2+0
The isomorphism between 24336bk1 and 121680dv1 is antisyplectic
12: 2=1+1
The isomorphism between 35574d2 and 5

The isomorphism between 25921a3 and 25921a4 is symplectic
The isomorphism between 25921a3 and 336973ba2 is symplectic
88: 3=3+0
The isomorphism between 286650je1 and 286650ol1 is symplectic
The isomorphism between 286650je1 and 286650om2 is symplectic
89: 3=3+0
The isomorphism between 22050dr2 and 22050dx2 is antisyplectic
90: 2=1+1
The isomorphism between 10816a1 and 54080f1 is antisyplectic
91: 2=1+1
The isomorphism between 10816a2 and 400192v2 is antisyplectic
92: 2=1+1
The isomorphism between 45486bn1 and 71478bv1 is antisyplectic
93: 2=1+1
The isomorphism between 5184u3 and 5184u4 is antisyplectic
The isomorphism between 5184u3 and 160704m2 is antisyplectic
94: 3=1+2
The isomorphism between 149093a2 and 356307e2 is symplectic
95: 2=2+0
The isomorphism between 2352r1 and 265776ci2 is symplectic
96: 2=2+0
The isomorphism between 2352r2 and 3920t2 is antisyplectic
The isomorphism between 2352r2 and 491568gq2 is antisyplectic
97: 3=1+2
The isomorphism between 10192bn1 and 30576cu1 is 

The isomorphism between 28224bn1 and 28224bn2 is symplectic
The isomorphism between 28224bn1 and 366912hl2 is symplectic
177: 3=3+0
The isomorphism between 28224bn3 and 28224bn4 is symplectic
The isomorphism between 28224bn3 and 366912hk2 is symplectic
178: 3=3+0
The isomorphism between 35378e1 and 389158k1 is antisyplectic
179: 2=1+1
The isomorphism between 7938m1 and 7938m2 is antisyplectic
The isomorphism between 7938m1 and 293706w2 is symplectic
180: 3=2+1
The isomorphism between 7938m3 and 7938m4 is antisyplectic
The isomorphism between 7938m3 and 246078m2 is symplectic
181: 3=2+1
The isomorphism between 247254bm2 and 412090b2 is antisyplectic
182: 2=1+1
The isomorphism between 7056bk2 and 35280ex2 is antisyplectic
183: 2=1+1
The isomorphism between 91728fk1 and 91728fl1 is symplectic
The isomorphism between 91728fk1 and 458640jr2 is symplectic
184: 3=3+0
The isomorphism between 3042b1 and 15210l1 is antisyplectic
185: 2=1+1
The isomorphism between 3042b2 and 112554f2 is antisyple

The isomorphism between 61866d2 and 67158r2 is symplectic
The isomorphism between 61866d2 and 158634p2 is symplectic
267: 3=3+0
The isomorphism between 151290bq1 and 211806cm1 is symplectic
268: 2=2+0
The isomorphism between 63504y1 and 317520hf2 is symplectic
269: 2=2+0
The isomorphism between 1521e1 and 235755q1 is symplectic
270: 2=2+0
The isomorphism between 144495n2 and 372645bi1 is symplectic
271: 2=2+0
The isomorphism between 1521e2 and 92781i2 is symplectic
272: 2=2+0
The isomorphism between 40768db1 and 122304fe1 is symplectic
273: 2=2+0
The isomorphism between 9408bp2 and 15680dg2 is antisyplectic
274: 2=1+1
The isomorphism between 35378k1 and 389158bg1 is antisyplectic
275: 2=1+1
The isomorphism between 8112ba2 and 164944e2 is symplectic
276: 2=2+0
The isomorphism between 270400ep1 and 270400gc1 is antisyplectic
277: 2=1+1
The isomorphism between 87318bf2 and 182574bp2 is antisyplectic
278: 2=1+1
The isomorphism between 100672y1 and 302016cp1 is antisyplectic
279: 2=1+1
The 

The isomorphism between 3610b2 and 107578e2 is antisyplectic
357: 2=1+1
The isomorphism between 78400p2 and 235200x2 is antisyplectic
358: 2=1+1
The isomorphism between 17298e2 and 294066h2 is antisyplectic
359: 2=1+1
The isomorphism between 1225b1 and 1225b2 is symplectic
The isomorphism between 1225b1 and 15925d2 is symplectic
The isomorphism between 1225b1 and 221725f1 is symplectic
360: 4=4+0
The isomorphism between 1225b3 and 1225b4 is symplectic
The isomorphism between 1225b3 and 15925h2 is symplectic
The isomorphism between 1225b3 and 221725e2 is symplectic
361: 4=4+0
The isomorphism between 68445bf2 and 150579m2 is symplectic
362: 2=2+0
The isomorphism between 2025f1 and 34425q1 is symplectic
363: 2=2+0
The isomorphism between 2025f2 and 22275j2 is symplectic
The isomorphism between 2025f2 and 99225bi1 is symplectic
364: 3=3+0
The isomorphism between 78650bz1 and 235950fx1 is antisyplectic
365: 2=1+1
The isomorphism between 5184bd3 and 5184bd4 is antisyplectic
The isomorphism b

In [101]:
assert len(isom_sets_red_split)==len(isom_sets_red_iso23)

In [102]:
set([tuple([len(s) for s in ss]) for ss in isom_sets_red_split])

{(1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (4, 0)}

In [103]:
[(s,len([ss for ss in isom_sets_red_split if tuple((len(si) for si in ss))==s])) for s in _]

[((1, 2), 15),
 ((3, 0), 56),
 ((3, 1), 10),
 ((2, 1), 30),
 ((2, 0), 142),
 ((2, 2), 2),
 ((1, 1), 121),
 ((4, 0), 8)]

In [106]:
[ss for ss in isom_sets_red_split if tuple(len(si) for si in ss)==(2,2)]

[[['162b1', '17334h1'], ['162b2', '293706ci2']],
 [['162c1', '293706x2'], ['162c2', '17334f1']]]

In [107]:
isrc2 = []
isrc3 = []
isrc6 = []
for ss in isom_sets_red_complete:
    i2 = any(EllipticCurve(s).isogenies_prime_degree(2) for s in ss)
    i3 = any(EllipticCurve(s).isogenies_prime_degree(3) for s in ss)
    if i2: isrc2.append(ss)
    if i3: isrc3.append(ss)
    if i2 and i3: isrc6.append(ss)
        #print ss
print("{} sets have 2-isogenies".format(len(isrc2)))
print("{} sets have 3-isogenies".format(len(isrc3)))
print("{} sets have both 2- and 3-isogenies".format(len(isrc6)))

38 sets have 2-isogenies
22 sets have 3-isogenies
0 sets have both 2- and 3-isogenies


In [110]:
s_filename = "mod7_red_symp_pairs.m"
a_filename = "mod7_red_anti_pairs.m"
ofile_s = open(s_filename, 'w')
ofile_a = open(a_filename, 'w')
ns = 0
na = 0
ofile_s.write('mod7_red_symp_pairs := [\\\n')
ofile_a.write('mod7_red_anti_pairs := [\\\n')
first_a = first_s = True
for ss in isom_sets_red_split:
    for s in ss:
        if s:
            s0 = s[0]
            for s1 in s[1:]:
                if not first_s:
                    ofile_s.write(", ")
                else:
                    first_s = False
                ofile_s.write('["{}", "{}"]\\\n'.format(s0,s1))
                ns += 1
    s0 = ss[0][0]
    if ss[1]:
        s1 = ss[1][0]
        if not first_a:
            ofile_a.write(", ")
        else:
            first_a = False
        ofile_a.write('["{}", "{}"]\\\n'.format(s0,s1))
        na += 1
ofile_s.write('];\n')
ofile_a.write('];\n')
ofile_s.close()
ofile_a.close()
print("{} pairs written to {}".format(ns,s_filename))
print("{} pairs written to {}".format(na,a_filename))

347 pairs written to mod7_red_symp_pairs.m
178 pairs written to mod7_red_anti_pairs.m


In [49]:
len(isom_sets_red_split)

384

In [44]:
t = polygen(QQ,'t')
E0 = KH0(t)
E1 = KH1(t)
E2 = KH2(t)

In [45]:
E0.discriminant().factor()

(-338688) * t * (t + 49/27)

In [36]:
E1.discriminant().factor()

(-432) * (t + 49/27) * t^2 * (t + 3)^7

In [37]:
E2.discriminant().factor()

(-122009559759792) * t * (t + 49/27)^2 * (t - 32/27)^7

In [43]:
E1.discriminant().gcd(E1.c6())

1

In [72]:
Ea0 = KH0(-3)
Ea0.label(), Ea0.j_invariant()

('294b1', -2401/6)

In [65]:
Ea2 = KH2(-3)
Ea2.label(), Ea2.j_invariant()

('678d2', -39934705050538129/2823126576537804)

In [66]:
Eb0 = KH0(32/27)
Eb0.label(), Eb0.j_invariant()

('2352l1', -2401/6)

In [67]:
Eb1 = KH1(32/27)
Eb1.label(), Eb1.j_invariant()

('5424l2', -39934705050538129/2823126576537804)

In [68]:
Eb0.is_quadratic_twist(Ea0)

-1

In [70]:
E0.j_invariant().factor()

(-38416/27) * t^-1 * (t + 49/27)^-1

### Now we reduce to a smaller sets  such that every congruent set is a twist of one in the smaller set

In [28]:
j_pairs_irred = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in isom_sets_irred]
j_sets_irred = list(Set(j_pairs_irred))
print("Out of {} irreducible sets, there are only {} distinct sets of j-invariants".format(len(j_pairs_irred),len(j_sets_irred)))

Out of 19883 irreducible sets, there are only 4939 distinct sets of j-invariants


In [29]:
j_pairs_red = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in isom_sets_red]
j_sets_red = list(Set(j_pairs_red))
print("Out of {} reducible sets, there are only {} distinct sets of j-invariants".format(len(j_pairs_red),len(j_sets_red)))

Out of 255 reducible sets, there are only 102 distinct sets of j-invariants


In [30]:
19883+255

20138

## Find all j-invariants of all curves (not just one per isogeny class) with congruences.  First, irreducible ones:

In [51]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [13]:
class_js('11a1')

{-122023936/161051, -4096/11, -52893159101157376/11}

In [14]:
set_js(isom_sets_irred[0])

{-8780800/2187, 270}

In [55]:
all_irred_js = sum([set_js(s) for s in isom_sets_irred], Set())

In [56]:
all_red_js_ss = sum([set_js(s) for s in isom_sets_red], Set())

In [57]:
all_red_js = sum([set_js(s) for s in isom_sets_red_complete], Set())

In [58]:
len(all_irred_js), len(all_red_js), len(all_red_js_ss)

(11761, 154, 382)

In [ ]:
"507b1", "78585b1"

In [61]:
test7iso(["507b1", "78585b1"])

[['507b1', '78585b1'], []]

In [62]:
E = EllipticCurve(j=0)

In [37]:
E.reducible_primes?

In [39]:
E.label()

'27a3'

In [44]:
[j for j in sum(j_sets_red, Set()) if j in cm_j_invariants(QQ)]

[-3375]

In [45]:
cm_j_invariants_and_orders(QQ)

[(-3, 3, -12288000),
 (-3, 2, 54000),
 (-3, 1, 0),
 (-4, 2, 287496),
 (-4, 1, 1728),
 (-7, 2, 16581375),
 (-7, 1, -3375),
 (-8, 1, 8000),
 (-11, 1, -32768),
 (-19, 1, -884736),
 (-43, 1, -884736000),
 (-67, 1, -147197952000),
 (-163, 1, -262537412640768000)]

In [76]:
len(isom_sets_irred)

19883

In [49]:
E = EllipticCurve(j=-32768)

In [50]:
E.label()

'121b1'

In [52]:
E.quadratic_twist(-11).label()

'121b2'

In [55]:
[s for s in isom_sets_irred if '9225a1' in s]

[['225a1', '9225a1', '9225e1', '11025c1']]

In [62]:
test_isom_labels('9225a1','11025c1', symplectic=False)

False

In [88]:
E = EllipticCurve('225a1')

In [69]:
EllipticCurve('225a1').is_quadratic_twist(EllipticCurve('11025c1'))

0

In [82]:
EllipticCurve('225a1').quadratic_twist(-3).label()

'225a2'

In [87]:
test_isom_labels('225a1','225a2',symplectic=False)

True

In [93]:
EllipticCurve('9225c1').j_invariant()

-1032125355/41

In [89]:
E.short_weierstrass_model()

Elliptic Curve defined by y^2 = x^3 + 80 over Rational Field

In [92]:
EllipticCurve([0,0,0,0,-28/80]).minimal_model().label()

'11025c1'

In [47]:
[test_isom(EllipticCurve([1,0]), EllipticCurve([(-4)*(-7)^k,0]), False) for k in range(4)]

[False, False, False, False]

In [40]:
test_isom?

In [87]:
sets2 = [s for s in isom_sets_irred if len(s)==2]
sets3 = [s for s in isom_sets_irred if len(s)==3]
sets4 = [s for s in isom_sets_irred if len(s)==4]
sets5 = [s for s in isom_sets_irred if len(s)==5]
len(sets2),len(sets3),len(sets4), len(sets5)

(18334, 1297, 246, 6)

In [78]:
def test4(s):
    js = [class_js(e) for e in s]
    #print(js)
    for i1 in range(len(js)):
        js1 = js[i1]
        for i2 in range(i1+1,len(js)):
            js2 = js[i2]
            #print(js1.intersection(js2))
            if len(js1.intersection(js2))>0: 
                print(s, js1, js2)

In [79]:
for s in sets4:
    test4(s)

(['225b1', '9225i1', '9225l1', '11025k1'], {0}, {0})
(['225b1', '9225i1', '9225l1', '11025k1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['3600be1', '147600dd1', '147600de1', '176400ip1'], {0}, {0})
(['3600be1', '147600dd1', '147600de1', '176400ip1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['3600bc1', '147600dp1', '147600dq1', '176400ke1'], {0}, {0})
(['3600bc1', '147600dp1', '147600dq1', '176400ke1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['225a1', '9225a1', '9225e1', '11025c1'], {0}, {0})
(['225a1', '9225a1', '9225e1', '11025c1'], {2813708206080/194754273881}, {2813708206080/194754273881})


In [68]:
E1 = EllipticCurve('225b1')
E2 = EllipticCurve('9225i1')
E3 = EllipticCurve('9225l1')
E4 = EllipticCurve('11025k1')
for E in [E1,E2,E3,E4]:
    print("{}: {}".format(E.label(), E.j_invariant()))
    print(E.galois_representation().image_type(7))

225b1: 0
The image is contained in the normalizer of a split Cartan group. (cm)
9225i1: 2813708206080/194754273881
The image is contained in the normalizer of a split Cartan group.
9225l1: 2813708206080/194754273881
The image is contained in the normalizer of a split Cartan group.
11025k1: 0


NotImplementedError: image_type is not implemented for cm-curves at bad prime.

In [72]:
E1.is_sextic_twist(E4)

-7/25

In [75]:
E1.short_weierstrass_model(), E4.short_weierstrass_model()

(Elliptic Curve defined by y^2 = x^3 + 10000 over Rational Field,
 Elliptic Curve defined by y^2 = x^3 - 2800 over Rational Field)

In [77]:
(-28).factor()

-1 * 2^2 * 7

In [83]:
for s in sets3:
    test4(s)

(['576g1', '65088cu1', '65088cv1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['288e1', '32544bd1', '32544be1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['576f1', '65088cm1', '65088cn1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['378225x1', '378225bm1', '378225bn1'], {2813708206080/194754273881}, {2813708206080/194754273881})
(['288a1', '32544b1', '32544c1'], {1372960580073984/235260548044817}, {1372960580073984/235260548044817})
(['378225bd1', '378225be1', '378225bq1'], {2813708206080/194754273881}, {2813708206080/194754273881})


In [85]:
E1 = EllipticCurve('378225bd1')
E2 = EllipticCurve('378225be1')
E3 = EllipticCurve('378225bq1')
[E.j_invariant() for E in [E1,E2,E3]]

[2813708206080/194754273881, 2813708206080/194754273881, 0]

In [86]:
E1.is_quadratic_twist(E2)

-3

In [88]:
for s in sets2: test4(s)

(['163072k1', '163072l1'], {-69832731032000/62748517}, {-69832731032000/62748517})
(['33075b1', '33075i1'], {0}, {0})
(['2700t1', '132300n1'], {0}, {0})
(['379456gs1', '379456gt1'], {16807000}, {16807000})
(['160083bs1', '160083bv1'], {0}, {0})
(['243a1', '11907g1'], {0}, {0})
(['151686m1', '151686n1'], {194967154845375/128}, {194967154845375/128})
(['44100u1', '44100y1'], {0}, {0})
(['4356a1', '213444dt1'], {0}, {0})
(['11907o1', '11907t1'], {0}, {0})
(['84672cr1', '84672cz1'], {0}, {0, -12288000})
(['297675ce1', '297675ct1'], {0}, {0})
(['176400kf1', '176400ky1'], {0, 54000}, {0})
(['160083bt1', '160083bw1'], {0, -12288000}, {0})
(['28224a1', '28224i1'], {0}, {0, 54000})
(['379456dd1', '379456de1'], {16807000}, {16807000})
(['298116bx1', '298116ce1'], {0}, {0, 54000})
(['3267d1', '160083bb1'], {0, -12288000}, {0})
(['2700l1', '132300bt1'], {0}, {0})
(['7056bb1', '7056bf1'], {0}, {0, 54000})
(['21168bp1', '21168bz1'], {0}, {0})
(['27a1', '1323m1'], {0, -12288000}, {0})
(['576a1', '282

(['298116br1', '298116cb1'], {0}, {0})
(['176400x1', '176400cx1'], {2268945/128}, {2268945/128})
(['298116bp1', '298116bv1'], {0}, {0})
(['132300du1', '132300es1'], {0}, {0})
(['190512dj1', '190512dn1'], {0}, {0})
(['382347bp1', '382347cb1'], {0}, {0})
(['176400iy1', '176400je1'], {0}, {0})
(['2450ba1', '2450bd1'], {2268945/128}, {2268945/128})
(['132300m1', '132300y1'], {0}, {0})
(['44100t1', '44100v1'], {0}, {0})
(['7803c1', '382347cd1'], {0}, {0})
(['190512df1', '190512du1'], {0}, {0})
(['3888p1', '190512v1'], {0}, {0})
(['1728v1', '84672iy1'], {0, -12288000}, {0})
(['5292h1', '5292j1'], {0}, {0})
(['47628a1', '47628j1'], {0}, {0})
(['11025j1', '11025l1'], {0}, {0})
(['189728bg1', '189728bh1'], {16807000}, {16807000})
(['6075a1', '297675cs1'], {0}, {0})
(['3267a1', '160083bq1'], {0}, {0})
(['432b1', '21168cz1'], {0}, {0})
(['297675ca1', '297675cn1'], {0}, {0})
(['28224cw1', '28224di1'], {0}, {0, 54000})
(['127449e1', '127449m1'], {0}, {0})
(['159201w1', '159201bf1'], {0}, {0})
(['13